# Assignment 1: Preprocessing and Text Classification

Student Name: Vaibhav Singh Makhloga

Student ID:1424591

# General Info

<b>Due date</b>: Wednesday, 20 March 2024 1pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day (both week and weekend days counted)

<b>Marks</b>: 9% of mark for class (with 8% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See "Using Jupyter Notebook and Python" page on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, Gensim, Keras and Pytorch. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3.8</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=https://www.nltk.org/book/>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place. In regards to the use of artificial intelligence tools in the context of academic integrity, please see the university's statement <a href="https://academicintegrity.unimelb.edu.au/plagiarism-and-collusion/artificial-intelligence-tools-and-technologies">here</a>.

# Overview

In this homework, you'll be working with a collection of tweets. The task is to predict the geolocation (country) where the tweet comes from. This homework involves writing code to preprocess data and perform text classification.

# Preprocessing (4 marks)

**Instructions**: Download the data (as1-data.json) from Canvas and put it in the same directory as this iPython notebook. Run the code below to load the json data. This produces two objects, `x` and `y`, which contains a list of  tweets and corresponding country labels (it uses the standard [2 letter country code](https://www.iban.com/country-codes)) respectively. **No implementation is needed.**

In [65]:
import json

x = []
y = []
data = json.load(open("as1-data.json"))
for k, v in data.items():
    x.append(k)
    y.append(v)
    
print("Number of tweets =", len(x))
print("Number of labels =", len(y))
print("\nSamples of data:")
for i in range(10):
    print("Country =", y[i], "\tTweet =", x[i])
    
assert(len(x) == 943)
assert(len(y) == 943)

Number of tweets = 943
Number of labels = 943

Samples of data:
Country = us 	Tweet = @Addictd2Success thx u for following
Country = us 	Tweet = Let's just say, if I were to ever switch teams, Khalesi would be top of the list. #girlcrush
Country = ph 	Tweet = Taemin jonghyun!!! Your birits make me go~ http://t.co/le8z3dntlA
Country = id 	Tweet = depart.senior 👻 rapat perdana (with Nyayu, Anita, and 8 others at Ruang Aescullap FK Unsri Madang) — https://t.co/swRALlNkrQ
Country = ph 	Tweet = Done with internship with this pretty little lady!  (@ Metropolitan Medical Center w/ 3 others) [pic]: http://t.co/1qH61R1t5r
Country = gb 	Tweet = Wow just Boruc's clanger! Haha Sunday League stuff that, Giroud couldn't believe his luck! #clown
Country = my 	Tweet = I'm at Sushi Zanmai (Petaling Jaya, Selangor) w/ 5 others http://t.co/bcNobykZ
Country = us 	Tweet = Mega Fest!!!! Its going down🙏🙌  @BishopJakes
Country = gb 	Tweet = @EllexxxPharrell wow love the pic babe xx
Country = us 	Tweet = You 

### Question 1 (1.0 mark)

**Instructions**: Next we need to preprocess the collected tweets to create a bag-of-words representation (based on frequency). The preprocessing steps required here are: (1) tokenize each tweet into individual word tokens (using NLTK `TweetTokenizer`); (2) lowercase all words; (3) remove any word that does not contain any English letters in the alphabet (e.g. {_hello_, _#okay_, _abc123_} would be kept, but not {_123_, _!!_}) and (4) remove stopwords (based on NLTK `stopwords`). An empty tweet (after preprocessing) and its country label should be **excluded** from the output (`x_processed` and `y_processed`).

**Task**: Complete the `preprocess_data(data, labels)` function. The function takes **a list of tweets** and **a corresponding list of country labels** as input, and returns **two lists**. For the first list, each element is a bag-of-words representation of a tweet (represented using a python dictionary). For the second list, each element is a corresponding country label. Note that while we do not need to preprocess the country labels (`y`), we need to have a new output list (`y_processed`) because some tweets maybe removed after the preprocessing (due to having an empty set of bag-of-words).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [66]:
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import re

tt = TweetTokenizer()
stopwords = set(stopwords.words('english')) #note: stopwords are all in lowercase

def preprocess_data(data, labels):
    
    ###
    x_processed = []
    y_processed = []
    
    for tweet, label in zip(data, labels):
        # To tokenize the tweets we use TweetTokenizer
        tokens = tt.tokenize(tweet)
        
        # this is done to filter words that do not contain english alphabets. Eg - ''#121'
        tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]
        
        # To remove stopwords from the dataset
        tokens = [token.lower() for token in tokens if token.lower() not in stopwords]
        
        # create BOG representation
        bag_of_words = {}
        for token in tokens:
            bag_of_words[token] = bag_of_words.get(token, 0) + 1
        
        # to not include empty tweets
        if bag_of_words:
            x_processed.append(bag_of_words)
            y_processed.append(label)

    return x_processed, y_processed
  
x_processed, y_processed = preprocess_data(x, y)

print("Number of preprocessed tweets =", len(x_processed))
print("Number of preprocessed labels =", len(y_processed))
print("\nSamples of preprocessed data:")
for i in range(10):
    print("Country =", y_processed[i], "\tTweet =", x_processed[i])

Number of preprocessed tweets = 943
Number of preprocessed labels = 943

Samples of preprocessed data:
Country = us 	Tweet = {'@addictd2success': 1, 'thx': 1, 'u': 1, 'following': 1}
Country = us 	Tweet = {"let's": 1, 'say': 1, 'ever': 1, 'switch': 1, 'teams': 1, 'khalesi': 1, 'would': 1, 'top': 1, 'list': 1, '#girlcrush': 1}
Country = ph 	Tweet = {'taemin': 1, 'jonghyun': 1, 'birits': 1, 'make': 1, 'go': 1, 'http://t.co/le8z3dntla': 1}
Country = id 	Tweet = {'depart.senior': 1, 'rapat': 1, 'perdana': 1, 'nyayu': 1, 'anita': 1, 'others': 1, 'ruang': 1, 'aescullap': 1, 'fk': 1, 'unsri': 1, 'madang': 1, 'https://t.co/swrallnkrq': 1}
Country = ph 	Tweet = {'done': 1, 'internship': 1, 'pretty': 1, 'little': 1, 'lady': 1, 'metropolitan': 1, 'medical': 1, 'center': 1, 'w': 1, 'others': 1, 'pic': 1, 'http://t.co/1qh61r1t5r': 1}
Country = gb 	Tweet = {'wow': 1, "boruc's": 1, 'clanger': 1, 'haha': 1, 'sunday': 1, 'league': 1, 'stuff': 1, 'giroud': 1, 'believe': 1, 'luck': 1, '#clown': 1}
Countr

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**For your testing**:

In [67]:
assert(len(x_processed) == len(y_processed))
assert(len(x_processed) > 800)

**Instructions**: Hashtags (i.e. topic tags which start with #) pose an interesting tokenisation problem because they often include multiple words written without spaces or capitalization. Run the code below to collect all unique hashtags in the preprocessed data. **No implementation is needed.**



In [68]:
def get_all_hashtags(data):
    hashtags = set([])
    for d in data:
        for word, frequency in d.items():
            if word.startswith("#") and len(word) > 1:
                hashtags.add(word)
    return hashtags

hashtags = get_all_hashtags(x_processed)
print("Number of hashtags =", len(hashtags))
print(sorted(hashtags))

Number of hashtags = 425
['#100percentpay', '#1stsundayofoctober', '#1yearofalmostisneverenough', '#2011prdctn', '#2015eebritishfilmacademyawards', '#2k16', '#2littlebirds', '#365picture', '#5sosacousticatlanta', '#5sosfam', '#8thannualpubcrawl', '#affsuzukicup', '#aflpowertigers', '#ahimacon14', '#aim20', '#airasia', '#allcity', '#alliswell', '#allwedoiscurls', '#amazing', '#anferneehardaway', '#ariona', '#art', '#arte', '#artwork', '#ashes', '#asian', '#asiangirl', '#askcrawford', '#askherforfback', '#askolly', '#asksteven', '#at', '#australia', '#awesome', '#awesomepict', '#barcelona', '#bart', '#bayofislands', '#beautiful', '#bedimages', '#bell', '#beringmy', '#bettybooppose', '#bff', '#big', '#bigbertha', '#bigbreakfast', '#blackhat', '#blessedmorethanicanimagine', '#blessedsunday', '#blogtourambiente', '#bluemountains', '#bonekachika', '#boomtaob', '#booyaa', '#bored', '#boredom', '#bradersisterhood', '#breaktime', '#breedingground', '#bringithomemy', '#brooksengland', '#burgers'

### Question 2 (1.0 mark)

**Instructions**: Our task here to tokenize the hashtags, by implementing the **MaxMatch algorithm** discussed in class.

NLTK has a list of words that you can use for matching, see starter code below (`words`). Be careful about efficiency with respect to doing word lookups. One extra challenge you have to deal with is that the provided list of words (`words`) includes only lemmas: your MaxMatch algorithm should match inflected forms by converting them into lemmas using the NLTK lemmatizer before matching (provided by the function `lemmatize(word)`). Note that the list of words (`words`) is the only source that you'll use for matching (i.e. you do not need to find  other external word lists). If you are unable to make any longer match, your code should default to matching a single letter.

For example, given "#newrecord", the algorithm should produce: \["#", "new", "record"\].

**Task**: Complete the `tokenize_hashtags(hashtags)` function by implementing the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [69]:
from nltk.corpus import wordnet
nltk.download('words')
nltk.download('wordnet')

lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
words = set(nltk.corpus.words.words()) #a list of words provided by NLTK
words = set([ word.lower() for word in words ]) #lowercase all the words for better matching



def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


def tokenize_hashtags(hashtags):
    hashtags_t = {}
    
    for hashtag in hashtags:
        tokens = []
        rest = hashtag
        
        while rest:
            
            # this is for finding the longest match
            found = False
            for i in range(len(rest), 0, -1):
                prefix = rest[:i]
                lemma_prefix = lemmatize(prefix)
                if lemma_prefix in words:
                    tokens.append(prefix)
                    rest = rest[i:]
                    found = True
                    break
                    
            # we take the single letter if no match is found
            if not found:
                tokens.append(rest[0])   
                rest = rest[1:]
        
        hashtags_t[hashtag] = tokens
    
    return tokenized_hashtags

#tokenise hashtags with MaxMatch
tokenized_hashtags = tokenize_hashtags(hashtags)

#print results
for k, v in sorted(tokenized_hashtags.items())[-30:]:
    print(k, v)

[nltk_data] Downloading package words to /Users/mac/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/mac/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'vega', 'n']
#veganfood ['#', 'vega', 'n', 'food']
#vegetables ['#', 'vegetable', 's']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'v', 's', 'c', 'o']
#vscocam ['#', 'v', 's', 'coca', 'm']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'wear', 'e', '9', '0', 's']
#wearesocial ['#', 'wear', 'es', 'o', 'c', 'i', 'al']
#white ['#', 'white']
#wings ['#', 'wing', 's']
#wok ['#', 'wo', 'k']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'mate', 's']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'ama', 's']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'w', 's', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [70]:
assert(len(tokenized_hashtags) == len(hashtags))
assert(tokenized_hashtags["#newrecord"] == ["#", "new", "record"])

### Question 3 (1.0 mark)

**Instructions**: Our next task is to tokenize the hashtags again, but this time using a **reversed version of the MaxMatch algorithm**, where matching begins at the end of the hashtag and progresses backwards (e.g. for <i>#helloworld</i>, we would process it right to left, starting from the last character <i>d</i>). Just like before, you should use the provided word list (`words`) for word matching.

**Task**: Complete the `tokenize_hashtags_rev(hashtags)` function by the MaxMatch algorithm. The function takes as input **a set of hashtags**, and returns **a dictionary** where key="hashtag" and value="a list of tokenised words".

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [71]:
def tokenize_hashtags_rev(hashtags):
    ###
    hashtags_tok_rev = {}
    
    for hashtag in hashtags:
        tokens = []
        rest = hashtag
        
        while rest:
            # This is again to find the longest string match
            found = False
            for i in range(len(rest), 0, -1):
                if rest[-i:] in words:
                    tokens.insert(0, rest[-i:])
                    rest = rest[:-i]
                    found = True
                    break
            
            # if we find no match then we have to select single letters
            if not found:
                tokens.insert(0, rest[-1])
                rest = rest[:-1]
        
        hashtags_tok_rev[hashtag] = tokens
    
    return hashtags_tok_rev

    
#tokenise hashtags with the reversed version of MaxMatch
tokenized_hashtags_rev = tokenize_hashtags_rev(hashtags)

#print results
for k, v in sorted(tokenized_hashtags_rev.items())[-30:]:
    print(k, v)

#vanilla ['#', 'vanilla']
#vca ['#', 'v', 'ca']
#vegan ['#', 'v', 'e', 'gan']
#veganfood ['#', 'v', 'e', 'gan', 'food']
#vegetables ['#', 'v', 'e', 'ge', 'tables']
#vegetarian ['#', 'vegetarian']
#video ['#', 'video']
#vma ['#', 'v', 'ma']
#voteonedirection ['#', 'vote', 'one', 'direction']
#vsco ['#', 'v', 's', 'c', 'o']
#vscocam ['#', 'v', 's', 'c', 'o', 'cam']
#walking ['#', 'walking']
#watch ['#', 'watch']
#weare90s ['#', 'we', 'are', '9', '0', 's']
#wearesocial ['#', 'we', 'are', 'social']
#white ['#', 'white']
#wings ['#', 'wing', 's']
#wok ['#', 'w', 'ok']
#wood ['#', 'wood']
#work ['#', 'work']
#workmates ['#', 'work', 'm', 'ates']
#world ['#', 'world']
#worldcup2014 ['#', 'world', 'cup', '2', '0', '1', '4']
#yellow ['#', 'yellow']
#yiamas ['#', 'y', 'i', 'a', 'mas']
#ynwa ['#', 'yn', 'wa']
#youtube ['#', 'you', 'tube']
#yummy ['#', 'yummy']
#yws13 ['#', 'y', 'w', 's', '1', '3']
#zweihandvollfarm ['#', 'z', 'wei', 'hand', 'vol', 'l', 'farm']


**For your testing:**

In [72]:
assert(len(tokenized_hashtags_rev) == len(hashtags))
assert(tokenized_hashtags_rev["#newrecord"] == ["#", "new", "record"])

### Question 4 (1.0 mark)

**Instructions**: The two versions of MaxMatch will produce different results for some of the hashtags. For a hastag that has different results, our task here is to use a **unigram language model** (lecture 3) to score them to see which is better. Recall that in a unigram language model we compute P(<i>#</i>, <i>hello</i>, <i>world</i> = P(<i>#</i>)\*P(<i>hello</i>)\*P(<i>world</i>).

You should: (1) use the NLTK's Brown corpus (`brown_words`) for collecting word frequencies (note: the words are already tokenised so no further tokenisation is needed); (2) lowercase all words in the corpus; (3) use add-one smoothing when computing the unigram probabilities; and (4) work in the log space to prevent numerical underflow.

**Task**: Build a unigram language model with add-one smoothing using the word counts from the Brown corpus. Iterate through the hashtags, and for each hashtag where MaxMatch and reversed MaxMatch produce different results, print the following: (1) the hashtag; (2) the results produced by MaxMatch and reversed MaxMatch; and (3) the log probability of each result as given by the unigram language model. Note: you **do not** need to print the hashtags where MaxMatch and reversed MaxMatch produce the same results.

An example output:
```
1. #abcd
MaxMatch = [#, a, bc, d]; LogProb = -2.3
Reversed MaxMatch = [#, a, b, cd]; LogProb = -3.5

2. #efgh
MaxMatch = [#, ef, g, h]; LogProb = -4.2
Reversed MaxMatch = [#, e, fgh]; LogProb = -3.1

```

Have a look at the output, and see if the sequences with better language model scores (i.e. less negative) are generally more coherent.

In [73]:
import nltk
from nltk.corpus import brown
import math

brown_words = brown.words()

# We need to lowercase words from the brown corpus
brown_words = [word.lower() for word in brown.words()]

# NOw we calculate word frequencies for the unigram model 
word_freq = nltk.FreqDist(brown_words)
total_words = len(brown_words)
vocab_size = len(word_freq)

# This is to do add-one smoothing 
word_freq_smooth = {word: (count + 1) / (total_words + vocab_size) for word, count in word_freq.items()}

#Function for the unigram model
def unigram_log_prob(words):
    log_prob = 0.0
    for word in words:
        #add-one smoothing for unknown words
        log_prob += math.log(word_freq_smooth.get(word, 1 / (total_words + vocab_size)))
    return log_prob

#Here we're gonna compare the maxmatch and maxmatch reverse tokens 
def compare_maxmatch(hashtags):
    for hashtag in hashtags:
        
        maxmatch_tokens = tokenize_hashtags([hashtag])[hashtag]
        rev_maxmatch_tokens = tokenize_hashtags_rev([hashtag])[hashtag]
        
        # This is to check if MaxMatch and reversed MaxMatch produce different results
        if maxmatch_tokens != rev_maxmatch_tokens:
            maxmatch_log_prob = unigram_log_prob(maxmatch_tokens)
            rev_maxmatch_log_prob = unigram_log_prob(rev_maxmatch_tokens)
            
            print(f"Hashtag: {hashtag}")
            print(f"MaxMatch: {maxmatch_tokens}; LogProb: {maxmatch_log_prob:.4f}")
            print(f"Reversed MaxMatch: {rev_maxmatch_tokens}; LogProb: {rev_maxmatch_log_prob:.4f}\n")


#Checking the final results
compare_maxmatch(hashtags)

Hashtag: #8thannualpubcrawl
MaxMatch: ['#', '8', 'than', 'nu', 'alp', 'u', 'b', 'crawl']; LogProb: -90.1049
Reversed MaxMatch: ['#', '8', 'th', 'annual', 'pub', 'crawl']; LogProb: -71.6476

Hashtag: #longranger
MaxMatch: ['#', 'long', 'ranger']; LogProb: -34.2982
Reversed MaxMatch: ['#', 'l', 'on', 'granger']; LogProb: -44.3785

Hashtag: #nickryrie
MaxMatch: ['#', 'nick', 'r', 'yr', 'ie']; LogProb: -63.1132
Reversed MaxMatch: ['#', 'nick', 'r', 'y', 'rie']; LogProb: -61.3214

Hashtag: #thenext5goldenyears
MaxMatch: ['#', 'then', 'ex', 't', '5', 'golden', 'year', 's']; LogProb: -81.0196
Reversed MaxMatch: ['#', 'the', 'next', '5', 'golden', 'year', 's']; LogProb: -62.5637

Hashtag: #thomassabo
MaxMatch: ['#', 'thomas', 'sab', 'o']; LogProb: -48.1169
Reversed MaxMatch: ['#', 'tho', 'mass', 'abo']; LogProb: -49.6410

Hashtag: #panthers
MaxMatch: ['#', 'panther', 's']; LogProb: -38.1922
Reversed MaxMatch: ['#', 'pant', 'hers']; LogProb: -39.1877

Hashtag: #cornell
MaxMatch: ['#', 'cornel',

# Text Classification (4 marks)

### Question 5 (1.0 mark)

**Instructions**: Here we are interested to do text classification, to predict the country of origin of a given tweet. The task here is to create training, development and test partitions from the preprocessed data (`x_processed`) and convert the bag-of-words representation into feature vectors.

**Task**: Create training, development and test partitions with a 70%/15%/15% ratio. Remember to preserve the ratio of the classes for all your partitions. That is, say we have only 2 classes and 70% of instances are labelled class A and 30% of instances are labelled class B, then the instances in training, development and test partitions should also preserve this 7:3 ratio. You may use sklearn's builtin functions for doing data partitioning.

Next, turn the bag-of-words dictionary of each tweet into a feature vector. You may also use sklearn's builtin functions for doing this (but if you don't want to use sklearn that's fine).

You should produce 6 objects: `x_train`, `x_dev`, `x_test` which contain the input feature vectors, and `y_train`, `y_dev` and `y_test` which contain the labels.

In [74]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split


x_train, x_dev, x_test = None, None, None
y_train, y_dev, y_test = None, None, None

###
# Convert bag-of-words representation to feature vectors
vectorizer = DictVectorizer()
X = vectorizer.fit_transform(x_processed)

# We're Splitting the data into training, development, and test sets 70%/15%/15% ratio. 
#Keeping the class ratio same as well. 
X_train, X_temp, y_train, y_temp = train_test_split(X, y_processed, test_size=0.3, random_state=42, stratify=y_processed)
X_dev, X_test, y_dev, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

# Converting back to arrays for consistency
x_train = X_train.toarray()
x_dev = X_dev.toarray()
x_test = X_test.toarray()

# Checking if it is done correctly
print("Training set shape:", x_train.shape)
print("Development set shape:", x_dev.shape)
print("Test set shape:", x_test.shape)


Training set shape: (660, 4387)
Development set shape: (141, 4387)
Test set shape: (142, 4387)


### Question 6 (1.0 mark)

**Instructions**: Now, let's build some classifiers. Here, we'll be comparing Naive Bayes and Logistic Regression. For each, you need to first find a good value for their main regularisation hyper-parameters, which you should identify using the scikit-learn docs or other resources. Use the development set you created for this tuning process; do **not** use cross-validation in the training set, or involve the test set in any way. You don't need to show all your work, but you do need to print out the **accuracy** with enough different settings to strongly suggest you have found an optimal or near-optimal choice. We should not need to look at your code to interpret the output. Your code should not take more than 1 minute to run (or penalty applies).

**Task**: Implement two text classifiers: Naive Bayes and Logistic Regression. Tune the hyper-parameters of these classifiers and print the task performance (accuracy) for different hyper-parameter settings.

In [75]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# This function is to train and evaluate classifiers with different hyperparameter settings
def tune_hyperparameters(classifier, hyperparameter_values, x_train, y_train, x_dev, y_dev):
    best_accuracy = 0.0
    best_hyperparameter = None
    
    for value in hyperparameter_values:
        # Set the hyperparameter value
        #for MultinomialNB we set the alpha value
        if isinstance(classifier, MultinomialNB):
            classifier.set_params(alpha=value)
            #For logistic we set the C value
        elif isinstance(classifier, LogisticRegression):
            classifier.set_params(C=value)
        
        # Training the classifier 
        classifier.fit(x_train, y_train)
        
        # Evaluation is done on the development set only
        y_pred = classifier.predict(x_dev)
        accuracy = accuracy_score(y_dev, y_pred)
        
    
        print(f"Hyperparameter = {value}, Accuracy = {accuracy:.4f}")
        
        # Finally, We Update the best hyperparameter if it gives us the best accuracy
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_hyperparameter = value
    
    return best_hyperparameter

# A set of standard hyparameters value to try 
alpha_values = [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
C_values = [0.01, 0.1, 1.0, 10.0, 100.0]

# Tuning hyperparameters for Naive Bayes
print("Tuning hyperparameters for Naive Bayes:")
nb_classifier = MultinomialNB()
best_alpha = tune_hyperparameters(nb_classifier, alpha_values, x_train, y_train, x_dev, y_dev)
print("Best alpha for Naive Bayes:", best_alpha)

# Tuning hyperparameters for Logistic Regression
print("\nTuning hyperparameters for Logistic Regression:")
lr_classifier = LogisticRegression(max_iter=1000)
best_C = tune_hyperparameters(lr_classifier, C_values, x_train, y_train, x_dev, y_dev)
print("Best C for Logistic Regression:", best_C)

Tuning hyperparameters for Naive Bayes:
Hyperparameter = 0.01, Accuracy = 0.2411
Hyperparameter = 0.1, Accuracy = 0.2553
Hyperparameter = 0.5, Accuracy = 0.2482
Hyperparameter = 1.0, Accuracy = 0.2340
Hyperparameter = 5.0, Accuracy = 0.2199
Hyperparameter = 10.0, Accuracy = 0.2340
Best alpha for Naive Bayes: 0.1

Tuning hyperparameters for Logistic Regression:
Hyperparameter = 0.01, Accuracy = 0.2199
Hyperparameter = 0.1, Accuracy = 0.2128
Hyperparameter = 1.0, Accuracy = 0.2270
Hyperparameter = 10.0, Accuracy = 0.2411
Hyperparameter = 100.0, Accuracy = 0.2411
Best C for Logistic Regression: 10.0


### Question 7 (1.0 mark)

**Instructions**: Using the best settings you have found, compare the two classifiers based on performance in the test set. Print out both **accuracy** and **macro-averaged F-score** for each classifier. Be sure to label your output. You may use sklearn's inbuilt functions.

**Task**: Compute test performance in terms of accuracy and macro-averaged F-score for both Naive Bayes and Logistic Regression, using their optimal hyper-parameter settings based on their development performance.

In [76]:
from sklearn.metrics import accuracy_score, f1_score

# Training Naive Bayes classifier with the best hyperparameter
nb_classifier = MultinomialNB(alpha=best_alpha)
nb_classifier.fit(x_train, y_train)

# Training Logistic Regression classifier with the best hyperparameter
lr_classifier = LogisticRegression(C=best_C, max_iter=1000)
lr_classifier.fit(x_train, y_train)

# We find predictions on the test set
nb_predictions = nb_classifier.predict(x_test)
lr_predictions = lr_classifier.predict(x_test)

# Computing accuracy and macro-averaged F-score for Naive Bayes & Logistic regression.
nb_accuracy = accuracy_score(y_test, nb_predictions)
nb_f1_macro = f1_score(y_test, nb_predictions, average='macro')

lr_accuracy = accuracy_score(y_test, lr_predictions)
lr_f1_macro = f1_score(y_test, lr_predictions, average='macro')


print("Performance on the test set:")
print("Naive Bayes Classifier:")
print(f"Accuracy: {nb_accuracy:.4f}")
print(f"Macro-averaged F-score: {nb_f1_macro:.4f}\n")

print("Logistic Regression Classifier:")
print(f"Accuracy: {lr_accuracy:.4f}")
print(f"Macro-averaged F-score: {lr_f1_macro:.4f}")


Performance on the test set:
Naive Bayes Classifier:
Accuracy: 0.2887
Macro-averaged F-score: 0.2835

Logistic Regression Classifier:
Accuracy: 0.2394
Macro-averaged F-score: 0.2246


### Question 8 (1.0 mark)

**Instructions**: Print the most important features and their weights for each class for the two classifiers.


**Task**: For each of the classifiers (Logistic Regression and Naive Bayes) you've built in the previous question, print out the top-20 features (words) with the highest weight for each class (countries).

An example output:
```
Classifier = Logistic Regression

Country = au
aaa (0.999) bbb (0.888) ccc (0.777) ...

Country = ca
aaa (0.999) bbb (0.888) ccc (0.777) ...

Classifier = Naive Bayes

Country = au
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...

Country = ca
aaa (-1.0) bbb (-2.0) ccc (-3.0) ...
```

Have a look at the output, and see if you notice any trend/pattern in the words for each country.

In [77]:
#Finding top features for lOGISTIC REGRESSION
print("Classifier = Logistic Regression")
print()
for i, country in enumerate(lr_classifier.classes_):
    print("Country =", country)
    top_features_idx = lr_classifier.coef_[i].argsort()[::-1][:20]
    top_features = [vectorizer.feature_names_[idx] for idx in top_features_idx]
    top_weights = lr_classifier.coef_[i][top_features_idx]
    for feature, weight in zip(top_features, top_weights):
        print(f"{feature} ({weight:.4f})", end=" ")
    print("\n")

# fINDING TOP FEATURES FOR NAIVE BAYES
print("Classifier = Naive Bayes")
print()
for i, country in enumerate(nb_classifier.classes_):
    print("Country =", country)
    top_features_idx = nb_classifier.feature_log_prob_[i].argsort()[::-1][:20]
    top_features = [vectorizer.feature_names_[idx] for idx in top_features_idx]
    top_log_probs = nb_classifier.feature_log_prob_[i][top_features_idx]
    for feature, log_prob in zip(top_features, top_log_probs):
        print(f"{feature} ({log_prob:.4f})", end=" ")
    print("\n")



Classifier = Logistic Regression

Country = au
australia (1.9312) #melbourne (1.8357) melbourne (1.7831) little (1.6712) @micksunnyg (1.6099) literally (1.5084) summerpoyi's (1.4988) https://t.co/7rcjjptvl7 (1.4988) @dasheryoung (1.4733) lachie (1.4733) @whennboys (1.3477) @jackgilinsky (1.3477) australian (1.3167) please (1.3035) http://t.co/blhj9cmxit (1.2570) @christorrano (1.2570) green (1.2226) ha (1.2147) beach (1.2073) @thomjrob (1.1496) 

Country = ca
@aliclarke (1.9057) happen (1.7706) hate (1.6019) thing (1.5207) gonna (1.5171) bed (1.4644) really (1.4508) @cheetahbiatch (1.4187) xoxo (1.4187) learning (1.3198) healthy (1.3031) @samanthasharris (1.2818) @lola9793 (1.2814) #nochillzone (1.2718) joking (1.2320) right (1.2267) cutest (1.1999) awesome (1.1433) think (1.1357) carlyle (1.1306) 

Country = de
roseninsel (1.7749) https://t.co/df7ficsci3 (1.7749) #utopia (1.4572) https://t.co/brkwmsvzrb (1.4572) jessica (1.4572) hyde (1.4572) gauting (1.4572) #truegrip (1.4572) done (